<a href="https://colab.research.google.com/github/faezsghfi/Comparison_of_LLMs_in_Persian_NLP/blob/main/NER/Llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### installing neccary package

In [ ]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes
!pip install hazm
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manyl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Preproccesing Data

In [ ]:
import hazm
from datasets import load_dataset
import ast

# Load dataset
dataset = load_dataset("persian_ner", split="test")

# Define tags and tag-to-index mappings
tags = ["O", "Event", "Facility", "LOC", "ORG", "PER", "Product", "Event", "Facility", "LOC", "ORG", "PER", "Product"]
tags_2_inex = {tag: i for i, tag in enumerate(tags)}
i_2_tags = {i: tag for i, tag in enumerate(tags)}

def get_predicted_list(sentence):
    # sentence = normalizer.normalize(sentence)

    start_index = sentence.find('[')
    if start_index == -1:
        return None
    end_index = sentence.find(']', start_index + 1)
    if end_index == -1:
        return None

    main_response = sentence[start_index + 1:end_index]
    list_of_tuples = ast.literal_eval(main_response)
    print(list_of_tuples)
    modified_tuples = [(word, tags_2_inex[tag]) for word, tag in list_of_tuples]
    predicted_tags_index = [t[1] for t in modified_tuples]
    return predicted_tags_index


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for persian_ner contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/persian_ner
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


### get Infernce

In [ ]:
def get_inference(prompt):
  messages = [
    {"role": "system", "content": ".تو یک دستیار ویرایشگر متن های فارسی هستی"},
    {"role": "user", "content": """{prompt}""".format(prompt = prompt)},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=2000,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  return(outputs[0]["generated_text"][len(prompt):])

### Prompting

In [ ]:
prompt_description = """ You need to label a given Persian token list with named entity labels.
    Named Entity Labels:

    PER (person)
    LOC (location)
    ORG (organization)
    Product (Product)
    Event (Event)
    Facility (Facility)

    Output Format:
    Your output format should be a list of tuples, where each tuple consists of a word from the input text and its corresponding named entity label.
    For words which are not part of any named entity, you should return 'O'.
    Do not say any sentence before list."""

### Main Loop

In [ ]:
import time

# golds = []
# preds = []
all_response = []

for i, data in enumerate(dataset):
    ts = data['tokens']
    ls = data['ner_tags']
    input_sentence = str(ts)

    prompt_arman = f"""
    Task Description:
    {prompt_description}

    Input:
    {input_sentence}
    """

    response = get_inference(prompt_arman)

    all_response.append(response)


    if i >= 200:
        break

    time.sleep(20)



In [ ]:
import numpy as np

np.save('preds_arman_response_en_prompt_llama3.out', np.array(all_response, dtype=object))

In [ ]:
import numpy as np
all_response = np.load('preds_arman_response_en_prompt_llama3.out.npy',allow_pickle=True)

In [ ]:
for indexxx , res in enumerate(all_response):
    all_response[indexxx] = all_response_raw[indexxx].replace(")\n(","),(")
    all_response[indexxx] = all_response[indexxx].replace("\n\n(","[(")
    all_response[indexxx] = all_response[indexxx].replace("('.') 'O')","('.','O')]")
    all_response[indexxx] = all_response[indexxx].replace("('.'), 'O')","('.','O')]")
    all_response[indexxx] = all_response[indexxx].replace("('.'),'O')","('.','O')]")
    # all_response[index] = all_response[index].replace(" (»', 'o')", " ('»', 'o')")
    start_index = all_response[indexxx] .find('[')
    if start_index == -1 :
        all_response[indexxx] = '[' + all_response[indexxx]
        start_index = all_response[indexxx] .find('[')
    end_index = all_response[indexxx] .find(']', start_index + 1)
    if end_index == -1 :
        all_response[indexxx] = all_response[indexxx] + ']'
        end_index = all_response[indexxx] .find(']', start_index + 1)
    main_response = all_response[indexxx] [start_index:end_index + 1]
    all_response[indexxx]  = main_response.lower()

In [ ]:
import ast

list_of_tuples = []
for i,res in enumerate(all_response):
    print(i)
    list_of_tuples.append(ast.literal_eval(res))

In [ ]:
tags = ["o", "event", "facility", "loc", "org", "per", "product", "event", "facility", "loc", "org", "per", "product"]
tags_2_inex = {tag: i for i, tag in enumerate(tags)}
i_2_tags = {i: tag for i, tag in enumerate(tags)}

In [ ]:
modified_tuples = []
for one_list in list_of_tuples:
    temp_list = []
    for tuple in one_list:
        # print(tuple[0],tuple[1])
        if tuple[1] in tags_2_inex:
            temp_list.append((tuple[0],tags_2_inex[tuple[1]]))
        else:
            temp_list.append((tuple[0],tags_2_inex['o']))
    modified_tuples.append(temp_list)

In [ ]:
predicted_tags_index = [t[1] for t in modified_tuples]

In [ ]:
preds = []
for t in modified_tuples:
    pred_tag = []
    for tag in t:
        pred_tag.append(tag[1])
    preds.append(pred_tag)

In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("persian_ner", split="test")[0:len(all_response)]

golds = []
for i, data in enumerate(dataset['ner_tags']):
    golds.append(data)

In [ ]:
dataset_tuple = []
for tokens,ner_tags in zip(dataset['tokens'],dataset['ner_tags']):
    tuples = []
    for i,j in zip(tokens,ner_tags):
        tuples.append((i,j))
    dataset_tuple.append(tuples)

In [ ]:
sorted_modified_tuples = []
sorted_dataset_tuples = []
for mt,dt in zip(modified_tuples,dataset_tuple):
    sorted_lists = sorted(zip(mt, dt), key=lambda x: x[0][0])
    sorted_mt, sorted_dt = zip(*sorted_lists)
    sorted_modified_tuples.append(sorted_mt)
    sorted_dataset_tuples.append(sorted_dt)

In [ ]:
print(len(sorted_modified_tuples[8]))
print(len(sorted_dataset_tuples[8]))

In [ ]:
prediction = []
for i in sorted_modified_tuples:
    for ii in i :
        prediction.append(ii[1])

In [ ]:
gold_label = []
for i in sorted_dataset_tuples:
    for ii in i :
        gold_label.append(ii[1])

### Evaluating

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Example prediction and label lists (2D arrays)
predictions = prediction
labels = gold_label

# Calculate precision, recall, and F1-score
precision = precision_score(labels, predictions, average='micro')
recall = recall_score(labels, predictions, average='micro')
f1 = f1_score(labels, predictions, average='micro')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)